In [ ]:
from scapy.all import rdpcap, bind_bottom_up, bind_layers, Raw
from scapy.contrib.coap import CoAP
from scapy.layers.inet import IP, UDP, TCP
from scapy.layers.dns import DNSRR
from scapy.contrib.mqtt import MQTT, MQTTSubscribe, MQTTConnect, MQTTTopicQOS, MQTTPublish
from scapy.contrib.mqttsn import MQTTSN, MQTTSNPublish


In [2]:
# Overwrite ports configuration for CoAP
bind_layers(UDP, CoAP, sport=5683)
bind_layers(UDP, CoAP, dport=5683)

In [3]:
# Overwrite ports configuration for MQTT
bind_layers(TCP, MQTT, sport=1883)
bind_layers(TCP, MQTT, dport=1883)

In [4]:
# Overwrite ports configuration for MQTT-SN
bind_bottom_up(UDP, MQTTSN, sport=1885)
bind_bottom_up(UDP, MQTTSN, dport=1885)
bind_layers(UDP, MQTTSN, dport=1885, sport=1885)

In [5]:
packets = rdpcap("./challenge2.pcapng")

## Question 1

How many different Confirmable PUT requests obtained an unsuccessful response from the local CoAP server?

In [31]:
# Set to track unique tokens of Confirmable PUT requests
confirmable_put_tokens = set()
unsuccessful_acks = 0

local_coap_server_ip = "127.0.0.1"

# Collect tokens of Confirmable PUT requests
for packet in packets:
    if (
        packet.haslayer(CoAP) # Check if packet has CoAP layer
        and packet[CoAP].code == 3  # PUT request
        and packet[CoAP].type == 0  # Confirmable
        and packet[IP].dst == local_coap_server_ip # Destination IP local server
        and packet[CoAP].token  # Ensure the token exists
    ):
        confirmable_put_tokens.add(packet[CoAP].token)

print(f"Unique Confirmable PUT request tokens: {len(confirmable_put_tokens)}")

not_found_tokens = set()
method_not_allowed_tokens = set()
# Count unsuccessful responses (ACKs) from the server
for packet in packets:
    if (
        packet.haslayer(CoAP) # Check if packet has CoAP layer
        and packet[CoAP].type == 2  # Acknowledgment response
        and packet[IP].src == local_coap_server_ip  # Response from server
        and packet[CoAP].token in confirmable_put_tokens  # Matches previous requests
        and packet[CoAP].code >= 128  # Error response (4.xx or 5.xx codes)
    ):
        unsuccessful_acks += 1
        print(f"Unsuccessful ACK response: {packet[CoAP].code} for token: {packet[CoAP].token}")
        if packet[CoAP].code == 132:
            not_found_tokens.add(packet[CoAP].token)
        elif packet[CoAP].code == 133:
            method_not_allowed_tokens.add(packet[CoAP].token)
        #print(packet[CoAP].token)
print(f"Tokens with 404 Not Found: {len(not_found_tokens)}")
print(f"Tokens with 405 Method Not Allowed: {len(method_not_allowed_tokens)}")
print(f"Number of unsuccessful ACK responses: {unsuccessful_acks}")

Unique Confirmable PUT request tokens: 26
Unsuccessful ACK response: 133 for token: b'r\xa5\xb8@\xc7\xf4\x08E'
Unsuccessful ACK response: 132 for token: b'\x9b\xe6p\x9d\x82\x05\x8e\xfc'
Unsuccessful ACK response: 133 for token: b'Y\xf2t\x1dW7\x9bT'
Unsuccessful ACK response: 132 for token: b'xX\x9c\xd0\x0c\xc1\xf6s'
Unsuccessful ACK response: 133 for token: b'\x9fC\xd8\n\x1f\xa8$I'
Unsuccessful ACK response: 133 for token: b'\xd7\xdbH\xb0x\x93Zg'
Unsuccessful ACK response: 132 for token: b'E\xca\xc1[\xab\xf2\xda,'
Unsuccessful ACK response: 133 for token: b'=\x9b\xaf\xceFAR\xc9'
Unsuccessful ACK response: 132 for token: b'\x86\xf0\x81>\xaf\xcf\xaf\xb0'
Unsuccessful ACK response: 133 for token: b'\xdd\x02P,\xe1\xce\x9c\x96'
Unsuccessful ACK response: 132 for token: b'\xa9\x19=\x83\x1f_\x04\xb8'
Unsuccessful ACK response: 132 for token: b'\x90\xba\x9ch\xc9\x11\x02"'
Unsuccessful ACK response: 133 for token: b'$\xcdi\x82\xb5P\x8e@'
Unsuccessful ACK response: 133 for token: b'W\n.\xcd\xb7/

Result: 22

## Question 2

How many CoAP resources in the coap.me public server received the same number of unique Confirmable and Non Confirmable GET requests?

In [26]:
 # Find IPs of the coap.me server by looking to the DNS requests
coapme = []
for p in packets:
    if (p.haslayer(DNSRR) # Only DNS Resource Record packets
        and p[DNSRR].type == 1 # Only records of type A
        # Only records for the coap.me server
        and p[DNSRR].rrname == b'coap.me.'
    ):
        # Save the IP of the coap.me server
        coapme.append(p[DNSRR].rdata)
# Only unique IPs
coapme = set(coapme)
print("coap.me IPs:", coapme)

coap.me IPs: {'134.102.218.18'}


In [27]:
con_requests = {}
non_con_requests = {}

for p in packets:
    if (
        p.haslayer(CoAP)  # Only CoAP Packets
        and p[CoAP].code == 1  # Only GET requests
        and p[IP].dst in coapme  # Requests directed to coap.me
    ):
        # Create a dictionary to store the count of each option
        option_counts = {}
        
        # Iterate over the options in the CoAP packet
        for option in p[CoAP].options:
            option_value = option[1]
            # Update the count of the option in the dictionary
            if option_value in option_counts:
                option_counts[option_value] += 1
            else:
                option_counts[option_value] = 1
        
        # Store the resource in the appropriate list based on message type
        if p[CoAP].type == 0:  # Confirmable
            # Update con_requests with option counts for each Confirmable message
            for opt, count in option_counts.items():
                if opt in con_requests:
                    con_requests[opt] += count
                else:
                    con_requests[opt] = count
        elif p[CoAP].type == 1:  # Non-Confirmable
            # Update non_con_requests with option counts for each Non-Confirmable message
            for opt, count in option_counts.items():
                if opt in non_con_requests:
                    non_con_requests[opt] += count
                else:
                    non_con_requests[opt] = count

matches = 0
for opt in con_requests:
    if opt in non_con_requests and con_requests[opt] == non_con_requests[opt] and len(opt) > 1:
        print(f"Option: {opt}, Confirmable Count: {con_requests[opt]}, Non-Confirmable Count: {non_con_requests[opt]}")
        matches += 1
        
print(f"Number of CoAP options with the same count for Confirmable and Non-Confirmable requests: {matches}")

Option: b'large', Confirmable Count: 14, Non-Confirmable Count: 14
Option: b'validate', Confirmable Count: 1, Non-Confirmable Count: 1
Option: b'secret', Confirmable Count: 1, Non-Confirmable Count: 1
Number of CoAP options with the same count for Confirmable and Non-Confirmable requests: 3


Result: 3

## Question 3

How many different MQTT clients subscribe to the public broker HiveMQ using multi-level wildcards?

In [32]:
# Find IPs of the HiveMQ server by looking to the DNS requests
hivemq = []
for p in packets:
    if (
        p.haslayer(DNSRR) # Only DNS Resource Record packets
        and p[DNSRR].type == 1 # Only records of type A
        # Only records for the HiveMQ server
        and p[DNSRR].rrname == b'broker.hivemq.com.'
    ):
        hivemq.append(p[DNSRR].rdata)
    
# Only unique IPs
hivemq = set(hivemq)
print("HiveMQ IPs:", hivemq)

HiveMQ IPs: {'18.192.151.104', '35.158.43.69'}


In [39]:
# Set to store unique source ports (clients)
unique_src_ports = set()
topic_src_ports = {}

for p in packets:
    if (
        p.haslayer(MQTT)  # Only MQTT packets
        and p[MQTT].type == 8  # Only Subscribe packets
        and p.haslayer(IP) # Only IP packets
        and p[IP].dst in hivemq  # Requests directed to HiveMQ
    ):

        found_topic = None 
        # Check if the packet has topics 
        if hasattr(p[MQTTSubscribe], "topics") and isinstance(p[MQTTSubscribe].topics, list):
            for topic_entry in p[MQTTSubscribe].topics:
                if hasattr(topic_entry, "topic") and topic_entry.topic:
                    found_topic = topic_entry.topic.decode() if isinstance(topic_entry.topic, bytes) else topic_entry.topic
                    break
                
        # Check if the packet has a Raw layer
        if not found_topic and p.haslayer(Raw): 
            raw_data = p[Raw].load
            found_topic = raw_data[:-1].decode()

        # Check if the topic contains multi-level wildcard 
        if found_topic and "#" in found_topic:
            topic_src_ports[found_topic] = topic_src_ports.get(found_topic, 0) + 1
            print(f"Topic: {found_topic}, Count: {topic_src_ports[found_topic]}, Source Port: {p[IP].sport}")
            unique_src_ports.add(p[IP].sport)

print(f"Number of unique source ports: {len(unique_src_ports)}")
print("Unique source ports:", unique_src_ports)


Topic: university/+/+/#, Count: 1, Source Port: 38641
Topic: university/room0/room1/#, Count: 1, Source Port: 38619
Topic: house/#, Count: 1, Source Port: 54449
Topic: university/#, Count: 1, Source Port: 38619
Topic: university/building2/section0/#, Count: 1, Source Port: 57863
Topic: factory/department3/floor0/#, Count: 1, Source Port: 38619
Number of unique source ports: 4
Unique source ports: {38641, 38619, 54449, 57863}


Result: 4

## Question 4

How many different MQTT clients specify a last Will Message to be directed to a topic having as first level ”university”?

In [147]:
mqtt_clients_with_lwt_university = set()

for p in packets:
    if (
        p.haslayer(MQTTConnect)  # Only MQTT Connect packets
        and p[MQTTConnect].willflag == 1  # Will flag is set
    ):
        # Check if willtopic exists and is not empty
        if hasattr(p[MQTTConnect], 'willtopic') and p[MQTTConnect].willtopic and len(p[MQTTConnect].willtopic) > 0:
            will_topic = p[MQTTConnect].willtopic.decode() if isinstance(p[MQTTConnect].willtopic, bytes) else p[MQTTConnect].willtopic
            print(f"Will topic: {will_topic}")

            # Check if the will topic contains "university"
            if "university" in will_topic:
                mqtt_clients_with_lwt_university.add(p[TCP].sport)
                print(f"Client (port source) with LWT topic containing 'university': {p[TCP].sport}")

print(f"Number of MQTT clients with LWT topic containing 'university': {len(mqtt_clients_with_lwt_university)}")

Will topic: university/department12/room1/temperature
Client (port source) with LWT topic containing 'university': 38083
Will topic: metaverse/room2/floor4
Will topic: metaverse/room2/room2
Number of MQTT clients with LWT topic containing 'university': 1


Result: I retrieved 4 packets, only one packet the number 4 (Source Port: 38083) has last Will Message directed to a topic having as first level ”university”

## Question 5

How many MQTT subscribers receive a last will message derived from a subscription without a wildcard?

In [ ]:
mqtt_lwt__without_wildcard = set()
mqtt_sub = set()

# Identify unique LWT topics without wildcard
for p in packets:
    if p.haslayer(MQTTConnect) and p[MQTTConnect].willflag == 1: # Only MQTT Connect packets with Will flag set
        if hasattr(p[MQTTConnect], 'willtopic') and p[MQTTConnect].willtopic:
            will_topic = p[MQTTConnect].willtopic.decode() if isinstance(p[MQTTConnect].willtopic, bytes) else p[MQTTConnect].willtopic
            # Check if the will topic does not contain wildcard
            if "#" not in will_topic and "+" not in will_topic:
                mqtt_lwt__without_wildcard.add(will_topic)

# Print the unique LWT topics without wildcard
print(f"Unique LWT topics without wildcard: {mqtt_lwt__without_wildcard}")
# Print the number of unique LWT topics without wildcard
print(f"Number of unique LWT topics without wildcard: {len(mqtt_lwt__without_wildcard)}")

# Count unique clients subscribing to these topics
for p in packets:
    if (
        p.haslayer(MQTTSubscribe) # Only MQTT Subscribe packets
    ):
    
        found_topic = None 
        # Check if the packet has topics 
        if hasattr(p[MQTTSubscribe], "topics") and isinstance(p[MQTTSubscribe].topics, list):
            for topic_entry in p[MQTTSubscribe].topics:
                if hasattr(topic_entry, "topic") and topic_entry.topic:
                    found_topic = topic_entry.topic.decode() if isinstance(topic_entry.topic, bytes) else topic_entry.topic
                    break
                
        # Check if the packet has a Raw layer
        if not found_topic and p.haslayer(Raw): 
            raw_data = p[Raw].load
            found_topic = raw_data[:-1].decode()
        
        # Check if the topic is in the set of unique LWT topics without wildcard
        if found_topic and found_topic in mqtt_lwt__without_wildcard:
            print(f"Will topic: {found_topic}")
            mqtt_sub.add(p[TCP].sport)
            print(f"Client (port source) with LWT topic without wildcard: {p[TCP].sport}")

# Count unique clients subscribing to these topics
print(f"Number of unique clients subscribing to LWT topics without wildcard: {len(mqtt_sub)}")


Unique LWT topics without wildcard: {'metaverse/room2/room2', 'university/department12/room1/temperature', 'metaverse/room2/floor4'}
Number of unique LWT topics without wildcard: 3
Will topic: university/department12/room1/temperature
Client (port source) with LWT topic without wildcard: 39551
Will topic: university/department12/room1/temperature
Client (port source) with LWT topic without wildcard: 53557
Will topic: university/department12/room1/temperature
Client (port source) with LWT topic without wildcard: 41789
Number of unique clients subscribing to LWT topics without wildcard: 3


Result: 3

## Question 6

How many MQTT publish messages directed to the public broker mosquitto are sent with the retain option and use QoS “At most once”?

In [47]:
# Find IPs of mosquitto server by looking to the DNS requests
mosquitto = []
for p in packets:
    if (
        p.haslayer(DNSRR) # Only DNS Resource Record packets
        and p[DNSRR].type == 1 # Only records of type A
        # Only records for the mosquitto server
        and p[DNSRR].rrname == b'test.mosquitto.org.'
    ):
        mosquitto.append(p[DNSRR].rdata)
    
# Only unique IPs
mosquitto = set(mosquitto)
print("mosquitto IPs:", mosquitto)

mosquitto IPs: {'5.196.78.28'}


In [ ]:
# List to store MQTT PUBLISH packets
mqtt_publish_with_conditions = []

for p in packets:
    if (
        p.haslayer(IP)  # Ensure IP layer exists
        and p.haslayer(MQTT)  # Ensure MQTT layer exists
        and p[IP].dst in mosquitto # IP address in mosquitto
        and p[MQTT].QOS == 0  # Ensure QoS is 0 (At most once)
        and p[MQTT].RETAIN == 1  # Ensure retain flag is set to 1
        and p[MQTT].type == 3  # Ensure message type is 3 (PUBLISH)
    ):
        mqtt_publish_with_conditions.append(p)

print(f"Number of MQTT PUBLISH packets with the specified conditions: {len(mqtt_publish_with_conditions)}")

Number of MQTT PUBLISH packets with the specified conditions: 208


Result: 208

## Question 7

How many MQTT-SN messages on port 1885 are sent by the clients to a broker in the local machine?

In [60]:
mqtt_sn_messages_to_local_broker = 0
local_broker_ip = "127.0.0.1"

for packet in packets:
    if packet.haslayer(UDP) and packet.haslayer(IP):  # Ensure both UDP and IP layers exist
        ip_dst = packet[IP].dst
        udp_layer = packet[UDP]
        
        if udp_layer.dport == 1885 and ip_dst == local_broker_ip:
            mqtt_sn_messages_to_local_broker += 1

print(f"7. MQTT-SN messages to local broker on port 1885: {mqtt_sn_messages_to_local_broker}")

7. MQTT-SN messages to local broker on port 1885: 0


Result: 0